# Run necessary background code

In [1]:
execfile('import_packages.py')

In [2]:
execfile('read_in_data.py')

# Functions and base values

In [54]:
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')

In [55]:
execfile('gen_baseline.py')
execfile('usercode_taxcalc.py')

In [56]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1045.477137,6301.956994,1125.229461,8675.639412,485.840762,828.509661,2014
1,1002.444226,6881.959962,1112.097673,9342.676993,555.954849,874.234922,2015
2,903.059951,7416.004365,931.573707,10017.083776,610.967737,824.155717,2016
3,936.312845,7839.731657,965.876547,10514.231722,647.634600,862.993269,2017
4,975.471545,8285.306731,1006.271668,11041.565089,684.443236,818.931279,2018
5,1010.804563,8747.330087,1042.720311,11589.956966,721.689235,800.194018,2019
6,1045.662400,9219.571782,1078.678771,12152.367907,758.733651,642.183000,2020
7,1085.048993,9700.098442,1119.308980,12726.309094,795.572002,728.105528,2021
8,1127.931552,10190.425253,1163.545539,13312.429537,832.767410,802.596534,2022
9,1172.719679,10695.375354,1209.747833,13916.168325,870.941407,864.518533,2023


In [57]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,273.627418,2588.712319,308.943861,6794.976567,160.711527,293.109821,2014
1,289.602189,2709.647188,340.890119,6963.817746,171.666565,324.629737,2015
2,295.171617,2861.364731,309.439238,7218.261731,183.732507,319.793554,2016
3,306.040564,3023.571922,320.833553,7469.385139,194.705193,337.446654,2017
4,318.839866,3195.418146,334.251531,7742.124214,205.771360,324.842875,2018
5,330.388716,3374.451290,346.358614,8027.528568,217.000372,319.973113,2019
6,341.782250,3558.521785,358.302874,8322.188650,228.206789,265.449731,2020
7,354.656040,3746.878744,371.798941,8624.413760,239.386229,296.759574,2021
8,368.672512,3939.800689,386.492922,8932.890713,250.694536,324.072325,2022
9,383.311832,4138.962941,401.839859,9250.127029,262.311139,347.093948,2023


In [58]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.02574,0.347,1312.193118,34.306208,828.509661,231.441998,2406.450986
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.02574,0.347,1353.612624,34.855388,874.234922,243.981131,2506.684065
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.02574,0.347,1230.342644,34.816996,824.155717,248.001094,2337.316451
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.02574,0.347,1274.287681,34.952204,862.993269,268.261581,2440.494735
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.02574,0.347,1308.734686,35.018974,818.931279,295.578104,2458.263042
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.02574,0.347,1373.991854,35.349483,800.194018,328.930336,2538.465691
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.02574,0.347,1504.774282,35.856932,642.183000,362.442904,2545.257118
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.02574,0.347,1528.831464,36.599743,728.105528,393.691242,2687.227978
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.02574,0.347,1567.950378,37.604625,802.596534,422.511919,2830.663456
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.02574,0.347,1606.388166,38.973402,864.518533,448.477548,2958.357649


# Specify economic and tax parameters

In [59]:
## Economic assumptions
p = 0.2     #real financial return
f = 0.32    #debt financing share
i_d = 0.068   #nominal interest rate on debt
pi = 0.024  #inflation rate
E = 0.058   #real return on equity

## Tax policy parameters
reform_start_year = 2017
tau_base = 0.35     #statutory tax rate, current law
tau_ref = 0.36      #statutory tax rate
progressive_rate = True
int_hc = 0.0        #haircut on interest deduction
## Modify bonus depreciation dataset to use new laws
bonus_data_ref = copy.deepcopy(bonus_data)
#for i in range(len(bonus_data_ref)):
#    if (bonus_data_ref['year'][i] >= reform_start_year):# and (bonus_data_ref['year'][i] < reform_start_year+5):
#        bonus_data_ref['bonus3'][i] = 1.
#        bonus_data_ref['bonus5'][i] = 1.
#        bonus_data_ref['bonus7'][i] = 1.
#        bonus_data_ref['bonus10'][i] = 1.
#        bonus_data_ref['bonus15'][i] = 1.
#        bonus_data_ref['bonus20'][i] = 1.
#        bonus_data_ref['bonus25'][i] = 1.
#        bonus_data_ref['bonus27'][i] = 1.
#        bonus_data_ref['bonus39'][i] = 1.

inventory_accounting = 0 #0 for historical mix of FIFO and LIFO, 1 for FIFO, 2 for LIFO, 3 for expensing
bonusrate_land = 0 #bonus depreciation rate for land. Does not affect model results

In [60]:
btax_dict1 = {
    2017: {
        'tau_c': 0.347,
        'tau_nc': 0.0,
        'tau_amt': 0.2,
        #'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        #'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        #'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        #'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        #'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        #'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        #'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        #'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        #'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        #'depr_land_bonus': 0.0,
        #'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    'undepBasis_corp_hc': {0: 0.0},
    'undepBasis_noncorp_hc': {0: 0.0},
    'amt_repeal': {9e99: False},
    'pymtc_repeal': {9e99: False},
    'ftc_hc': {9e99: 0.0},
    'sec199_hc': {9e99: 0.0},
    'oldIntPaid_corp_hc': {0: 0.0},
    'newIntPaid_corp_hc': {0: 0.0},
    'netIntPaid_corp_hc': {0: 0.0},
    'oldIntPaid_noncorp_hc': {0: 0.0},
    'newIntPaid_noncorp_hc': {0: 0.0}
}
iit_params_ref = {
    reform_start_year: {
        '_II_rt7': [0.396]
    }
}
execfile('reform_implementation.py')

# Calculate the investment response

In [61]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': 0.0,
    'inv_usercost_nc': 0.0,
    'inv_eatr': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': 0.0
}
execfile('responses.py')

In [62]:
response_results = investmentResponse(reform_start_year)

***** RUN 2014 *****
***** RUN 2015 *****
***** RUN 2016 *****
***** RUN 2017 *****
***** RUN 2018 *****
***** RUN 2019 *****
***** RUN 2020 *****
***** RUN 2021 *****
***** RUN 2022 *****
***** RUN 2023 *****
***** RUN 2024 *****
***** RUN 2025 *****
***** RUN 2026 *****
***** RUN 2027 *****


In [63]:
response_results

,Asset,deltaIc2014,deltaEc2014,deltaInc2014,deltaEnc2014,deltaIc2015,deltaEc2015,deltaInc2015,deltaEnc2015,deltaIc2016,...,deltaInc2025,deltaEnc2025,deltaIc2026,deltaEc2026,deltaInc2026,deltaEnc2026,deltaIc2027,deltaEc2027,deltaInc2027,deltaEnc2027
0,Aerospace products and parts manufacturing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Air transportation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aircraft,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"All other nonmanufacturing, n.e.c.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Amusement and recreation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Autos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Books,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Chemical manufacturing, ex. pharma and med",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Communication,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Communications,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Run other responses and recalculate corporate tax revenue

In [ ]:
execfile('gen_reform.py')

### Check capital results

In [67]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1045.477137,6301.956994,1125.229461,8675.639412,485.840762,828.509661,2014
1,1002.444226,6881.959962,1112.097673,9342.676993,555.954849,874.234922,2015
2,903.059951,7416.004365,931.573707,10017.083776,610.967737,824.155717,2016
3,936.312845,7839.731657,965.876547,10514.231722,647.634600,862.993269,2017
4,975.471545,8285.306731,1006.271668,11041.565089,684.443236,818.931279,2018
5,1010.804563,8747.330087,1042.720311,11589.956966,721.689235,800.194018,2019
6,1045.662400,9219.571782,1078.678771,12152.367907,758.733651,642.183000,2020
7,1085.048993,9700.098442,1119.308980,12726.309094,795.572002,728.105528,2021
8,1127.931552,10190.425253,1163.545539,13312.429537,832.767410,802.596534,2022
9,1172.719679,10695.375354,1209.747833,13916.168325,870.941407,864.518533,2023


In [68]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1045.477137,6301.956994,1125.229461,8675.639412,485.840762,828.509661,2014
1,1002.444226,6881.959962,1112.097673,9342.676993,555.954849,874.234922,2015
2,903.059951,7416.004365,931.573707,10017.083776,610.967737,824.155717,2016
3,936.312845,7839.731657,965.876547,10514.231722,647.634600,862.993269,2017
4,975.471545,8285.306731,1006.271668,11041.565089,684.443236,818.931279,2018
5,1010.804563,8747.330087,1042.720311,11589.956966,721.689235,800.194018,2019
6,1045.662400,9219.571782,1078.678771,12152.367907,758.733651,642.183000,2020
7,1085.048993,9700.098442,1119.308980,12726.309094,795.572002,728.105528,2021
8,1127.931552,10190.425253,1163.545539,13312.429537,832.767410,802.596534,2022
9,1172.719679,10695.375354,1209.747833,13916.168325,870.941407,864.518533,2023


In [69]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,273.627418,2588.712319,308.943861,6794.976567,160.711527,293.109821,2014
1,289.602189,2709.647188,340.890119,6963.817746,171.666565,324.629737,2015
2,295.171617,2861.364731,309.439238,7218.261731,183.732507,319.793554,2016
3,306.040564,3023.571922,320.833553,7469.385139,194.705193,337.446654,2017
4,318.839866,3195.418146,334.251531,7742.124214,205.771360,324.842875,2018
5,330.388716,3374.451290,346.358614,8027.528568,217.000372,319.973113,2019
6,341.782250,3558.521785,358.302874,8322.188650,228.206789,265.449731,2020
7,354.656040,3746.878744,371.798941,8624.413760,239.386229,296.759574,2021
8,368.672512,3939.800689,386.492922,8932.890713,250.694536,324.072325,2022
9,383.311832,4138.962941,401.839859,9250.127029,262.311139,347.093948,2023


In [70]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,273.627418,2588.712319,308.943861,6794.976567,160.711527,293.109821,2014
1,289.602189,2709.647188,340.890119,6963.817746,171.666565,324.629737,2015
2,295.171617,2861.364731,309.439238,7218.261731,183.732507,319.793554,2016
3,306.040564,3023.571922,320.833553,7469.385139,194.705193,337.446654,2017
4,318.839866,3195.418146,334.251531,7742.124214,205.771360,324.842875,2018
5,330.388716,3374.451290,346.358614,8027.528568,217.000372,319.973113,2019
6,341.782250,3558.521785,358.302874,8322.188650,228.206789,265.449731,2020
7,354.656040,3746.878744,371.798941,8624.413760,239.386229,296.759574,2021
8,368.672512,3939.800689,386.492922,8932.890713,250.694536,324.072325,2022
9,383.311832,4138.962941,401.839859,9250.127029,262.311139,347.093948,2023


### Check combined results

In [73]:
combined_ref

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


,year,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,2014,2406.450986,828.509661,231.441998,34.306208,1312.193118,0.02574,0.347,421.555664,101.582376,4.457941,3.700228,320.731
1,2015,2506.684065,874.234922,243.981131,34.855388,1353.612624,0.02574,0.347,434.862110,91.794692,4.529305,3.799722,343.797
2,2016,2337.316451,824.155717,248.001094,34.816996,1230.342644,0.02574,0.347,395.260349,96.318143,4.524316,3.895522,299.571
3,2017,2440.494735,862.993269,268.261581,34.952204,1274.287681,0.02574,0.347,409.378148,99.935946,4.541886,3.978088,310.006
4,2018,2458.263042,818.931279,295.578104,35.018974,1308.734686,0.02574,0.347,420.444606,96.600049,4.550562,4.052119,324.343
5,2019,2538.465691,800.194018,328.930336,35.349483,1373.991854,0.02574,0.347,441.409149,97.986091,4.593510,4.117569,343.899
6,2020,2545.257118,642.183000,362.442904,35.856932,1504.774282,0.02574,0.347,483.424362,103.553749,4.659451,4.180064,380.350
7,2021,2687.227978,728.105528,393.691242,36.599743,1528.831464,0.02574,0.347,491.152982,109.003947,4.755976,4.243011,382.662
8,2022,2830.663456,802.596534,422.511919,37.604625,1567.950378,0.02574,0.347,503.720339,114.806528,4.886556,4.310368,389.490
9,2023,2958.357649,864.518533,448.477548,38.973402,1606.388166,0.02574,0.347,516.068877,121.619274,5.064423,4.386026,395.128


In [74]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.02574,0.347,1312.193118,34.306208,828.509661,231.441998,2406.450986
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.02574,0.347,1353.612624,34.855388,874.234922,243.981131,2506.684065
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.02574,0.347,1230.342644,34.816996,824.155717,248.001094,2337.316451
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.02574,0.347,1274.287681,34.952204,862.993269,268.261581,2440.494735
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.02574,0.347,1308.734686,35.018974,818.931279,295.578104,2458.263042
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.02574,0.347,1373.991854,35.349483,800.194018,328.930336,2538.465691
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.02574,0.347,1504.774282,35.856932,642.183000,362.442904,2545.257118
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.02574,0.347,1528.831464,36.599743,728.105528,393.691242,2687.227978
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.02574,0.347,1567.950378,37.604625,802.596534,422.511919,2830.663456
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.02574,0.347,1606.388166,38.973402,864.518533,448.477548,2958.357649


In [75]:
compare = combined_ref['taxrev'] - combined_base['taxrev']
print ("Change in corporate tax revenue, " + str(reform_start_year) + "-" + str(reform_start_year + 9) + ": " +
       str(int((sum(combined_ref['taxrev'][reform_start_year-2014:reform_start_year-2004]) - 
           sum(combined_base['taxrev'][reform_start_year-2014:reform_start_year-2004])) * 100.) / 100.) + 
       " billion dollars")

Change in corporate tax revenue, 2017-2026: 0.0 billion dollars


In [76]:
for i in compare:
    print i

-1.13686837722e-13
0.0
-2.27373675443e-13
5.68434188608e-14
-5.68434188608e-14
-1.13686837722e-13
-1.70530256582e-13
-5.68434188608e-14
-5.68434188608e-14
-1.13686837722e-13
-5.68434188608e-14
0.0
-1.13686837722e-13
-2.27373675443e-13


## Calculate percent changes in investment income and pass-through income

In [77]:
execfile('multipliers_for_taxcalc.py')

In [78]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1125.229461,2014,1368.437226,1368.437226,1.0,1.0,1.0,1.0,1.0,1.0
1,1112.097673,2015,1362.951085,1362.951085,1.0,1.0,1.0,1.0,1.0,1.0
2,931.573707,2016,1178.776620,1178.776620,1.0,1.0,1.0,1.0,1.0,1.0
3,965.876547,2017,1214.592554,1214.592554,1.0,1.0,1.0,1.0,1.0,1.0
4,1006.271668,2018,1153.898702,1153.898702,1.0,1.0,1.0,1.0,1.0,1.0
5,1042.720311,2019,1143.947120,1143.947120,1.0,1.0,1.0,1.0,1.0,1.0
6,1078.678771,2020,1043.730562,1043.730562,1.0,1.0,1.0,1.0,1.0,1.0
7,1119.308980,2021,1115.302734,1115.302734,1.0,1.0,1.0,1.0,1.0,1.0
8,1163.545539,2022,1185.894127,1185.894127,1.0,1.0,1.0,1.0,1.0,1.0
9,1209.747833,2023,1243.810693,1243.810693,1.0,1.0,1.0,1.0,1.0,1.0


## Pass changes to Tax-Calculator

In [79]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [80]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

0.0
6.63697694599e-14
6.46208859543e-14
-8.90400431545e-14
5.2461623269e-14
6.66282325216e-14
0.0
7.77076601943e-14
-2.50869831809e-14
8.42405506971e-14
1.01928397244e-14
0.0
8.96756668808e-14
1.73996650506e-13
4.40776197458e-13


In [81]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,0.000000e+00,2014,-1.136868e-13,-1.136868e-13
1,6.636977e-14,2015,0.000000e+00,6.636977e-14
2,6.462089e-14,2016,-2.273737e-13,-1.627528e-13
3,-8.904004e-14,2017,5.684342e-14,-3.219662e-14
4,5.246162e-14,2018,-5.684342e-14,-4.381796e-15
5,6.662823e-14,2019,-1.136868e-13,-4.705861e-14
6,0.000000e+00,2020,-1.705303e-13,-1.705303e-13
7,7.770766e-14,2021,-5.684342e-14,2.086424e-14
8,-2.508698e-14,2022,-5.684342e-14,-8.193040e-14
9,8.424055e-14,2023,-1.136868e-13,-2.944629e-14


In [82]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

-3.21966242937e-14
-4.38179559177e-15
-4.70586052e-14
-1.70530256582e-13
2.08642413335e-14
-8.19304020417e-14
-2.94462870245e-14
-4.66505791364e-14
0.0
-2.40111708409e-14
-5.3377024937e-14


-4.15341479378e-13
